<a href="https://colab.research.google.com/github/Rahulraj31/Fake-News-LSTM/blob/main/Fake_News_Classifier_Using_LSTM.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>


# Fake News Classifier Using LSTM


Dataset: https://www.kaggle.com/c/fake-news/data#

In [1]:
import zipfile

local_zip = '/content/drive/MyDrive/Colab Notebooks/fake-news.zip'
zip_ref = zipfile.ZipFile(local_zip, 'r')
zip_ref.extractall('/content')
zip_ref.close()

In [2]:
import pandas as pd
df= pd.read_csv("/content/train.csv")

In [3]:
df.head()

,id,title,author,text,label
0,0,House Dem Aide: We Didn’t Even See Comey’s Let...,Darrell Lucus,House Dem Aide: We Didn’t Even See Comey’s Let...,1
1,1,"FLYNN: Hillary Clinton, Big Woman on Campus - ...",Daniel J. Flynn,Ever get the feeling your life circles the rou...,0
2,2,Why the Truth Might Get You Fired,Consortiumnews.com,"Why the Truth Might Get You Fired October 29, ...",1
3,3,15 Civilians Killed In Single US Airstrike Hav...,Jessica Purkiss,Videos 15 Civilians Killed In Single US Airstr...,1
4,4,Iranian woman jailed for fictional unpublished...,Howard Portnoy,Print \nAn Iranian woman has been sentenced to...,1


**1: unreliable**<br>

**0: reliable**

In [4]:
df.shape

(20800, 5)

In [5]:
df.isnull().sum()

id           0
title      558
author    1957
text        39
label        0
dtype: int64

In [6]:
df=df.dropna()

In [7]:
# Get the Independent and Dependent Features
X=df.drop('label',axis=1)
Y=df['label']

In [8]:
X.shape,Y.shape

((18285, 4), (18285,))

In [9]:
from tensorflow.keras.layers import Embedding
from tensorflow.keras.preprocessing.sequence import pad_sequences  #for proper same size input to embedding layer, same as widow in genshim
from tensorflow.keras.models import Sequential
from tensorflow.keras.preprocessing.text import one_hot
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense,Dropout

In [10]:
# vocabulary size
voc_size =5000

## One hot Representation


In [11]:
messages=X.copy()

In [12]:
messages.reset_index(inplace=True)

In [13]:
import nltk
import re
from nltk.corpus import stopwords

In [14]:
nltk.download('stopwords')

[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


True

In [15]:
# Dataset preprocessing
from nltk.stem.porter import PorterStemmer
ps= PorterStemmer()
corpus = []
for i in range(0,len(messages)):
  review=re.sub('[^a-zA-Z]',' ',messages["title"][i]) #substitute everthing with blank apart from letters a-z and A-Z in title
  
  review =review.lower() #convert to lower case
  
  review=review.split() 

  review = [ps.stem(word) for word in review if not word in stopwords.words('english')] #stems words no present in stopword
  review = ' '.join(review) #combining list of words to a list of sentences
  corpus.append(review)
  

In [16]:
onehot_rep= [one_hot(words,voc_size) for words in corpus]

In [17]:
# one hot for first 5 sentences
onehot_rep[0:5]

[[526, 1902, 3027, 1278, 4250, 3557, 2260, 73, 2124, 3784],
 [2897, 1113, 1163, 4081, 4330, 4419, 2778],
 [2149, 2218, 3497, 1515],
 [363, 3254, 2960, 2075, 3718, 4915],
 [3317, 4330, 2789, 1120, 2113, 2731, 4330, 1949, 3996, 3289]]

## Embedding Representation

In [18]:
sent_length =20
embedded_docs=pad_sequences(onehot_rep,padding='pre',maxlen=sent_length)
print(embedded_docs)

[[   0    0    0 ...   73 2124 3784]
 [   0    0    0 ... 4330 4419 2778]
 [   0    0    0 ... 2218 3497 1515]
 ...
 [   0    0    0 ... 1501 1886 2864]
 [   0    0    0 ... 4729  623 2748]
 [   0    0    0 ... 1707 4393 3302]]


In [19]:
embedded_docs[1]

array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0, 2897, 1113, 1163, 4081, 4330, 4419, 2778], dtype=int32)

In [20]:
# creating model
embedding_vector_features =40   #dimwnsion of feature vector of embedding 
model=Sequential()

model.add(Embedding(voc_size,embedding_vector_features,input_length=sent_length))
model.add(Dropout(0.5))
model.add(LSTM(100))  # 1 layer 100 neuron
model.add(Dropout(0.3))
model.add(Dense(1,activation='sigmoid'))


In [21]:
model.compile(loss='binary_crossentropy',optimizer='adam',metrics=['accuracy'])

In [22]:
model.summary()

Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding (Embedding)        (None, 20, 40)            200000    
_________________________________________________________________
dropout (Dropout)            (None, 20, 40)            0         
_________________________________________________________________
lstm (LSTM)                  (None, 100)               56400     
_________________________________________________________________
dropout_1 (Dropout)          (None, 100)               0         
_________________________________________________________________
dense (Dense)                (None, 1)                 101       
Total params: 256,501
Trainable params: 256,501
Non-trainable params: 0
_________________________________________________________________


In [23]:
import numpy as np
X_final=np.array(embedded_docs)
y_final=np.array(Y)

In [24]:
from sklearn.model_selection import train_test_split
X_train, X_val, y_train, y_val = train_test_split(X_final, y_final, test_size=0.33, random_state=42)


In [25]:
model.fit(X_train,y_train,validation_data=(X_val,y_val),epochs=10,batch_size=32)


Epoch 1/10
383/383 [==============================] - 6s 9ms/step - loss: 0.2888 - accuracy: 0.8682 - val_loss: 0.2074 - val_accuracy: 0.9140
Epoch 2/10
383/383 [==============================] - 3s 7ms/step - loss: 0.1506 - accuracy: 0.9413 - val_loss: 0.1973 - val_accuracy: 0.9157
Epoch 3/10
383/383 [==============================] - 3s 8ms/step - loss: 0.1175 - accuracy: 0.9549 - val_loss: 0.2145 - val_accuracy: 0.9127
Epoch 4/10
383/383 [==============================] - 3s 7ms/step - loss: 0.0912 - accuracy: 0.9651 - val_loss: 0.2283 - val_accuracy: 0.9084
Epoch 5/10
383/383 [==============================] - 3s 7ms/step - loss: 0.0703 - accuracy: 0.9743 - val_loss: 0.2567 - val_accuracy: 0.9004
Epoch 6/10
383/383 [==============================] - 3s 8ms/step - loss: 0.0549 - accuracy: 0.9799 - val_loss: 0.3216 - val_accuracy: 0.9171
Epoch 7/10
383/383 [==============================] - 3s 7ms/step - loss: 0.0421 - accuracy: 0.9861 - val_loss: 0.3206 - val_accuracy: 0.9180
Epoch 


## Performance Metrics And Accuracy

In [26]:
y_pred=model.predict_classes(X_val)

/usr/local/lib/python3.7/dist-packages/tensorflow/python/keras/engine/sequential.py:455: UserWarning: `model.predict_classes()` is deprecated and will be removed after 2021-01-01. Please use instead:* `np.argmax(model.predict(x), axis=-1)`,   if your model does multi-class classification   (e.g. if it uses a `softmax` last-layer activation).* `(model.predict(x) > 0.5).astype("int32")`,   if your model does binary classification   (e.g. if it uses a `sigmoid` last-layer activation).
  warnings.warn('`model.predict_classes()` is deprecated and '


In [27]:
from sklearn.metrics import confusion_matrix
from sklearn.metrics import accuracy_score
confusion_matrix(y_val,y_pred)

array([[3096,  323],
       [ 196, 2420]])

In [28]:
accuracy_score(y_val,y_pred)

0.9140016570008285